## Imports

In [1]:
import numpy as np

## Network Class

In [30]:
"""
Simple_NN
Class to create a simple neural network
    Learning Algorithm: stochastic gradient descent with backpropagation
    Activation Function: sigmoid
    Cost Function: MSE
"""
class Simple_NN(object):
    """ 
    INITIALIZE THE NETWORK
    """
    def __init__(self, layers, activation_function="sigmoid", cost_function="MSE"):
        """
        self.layers is a list of numbers where the ith number how many neurons are in
        the ith layer of the network.
        """
        self.layers = layers;
        self.num_layers = len(layers);
        
        """
        self.weights[Layer - 1, input_neuron, output_neuron] = 
                                            List of weight matrices for each layer.                      
        self.biases[Layer - 1, neuron] = 
                                            List of vectors with biases for each neuron  
        FOR EXAMPLE:
            self.weights[layer, j, i] = weight going into the jth neuron of the lth layer
                                    from the ith neuron of the (l-1)st layer 
            self.biases[layer, k] = bias on the kth nuron of the lth layer
        NOTE: layer 0 is the input layer, so self.weights[0] is the weights going into layer 1
        """
        self.weights = [];
        self.biases = [];
        self.Z = [];
        self.activations = [];
        # Create matrices with correct dimensions 
        for layer_num in range(1, self.num_layers):
            self.weights.append(np.random.randn(layers[layer_num], layers[layer_num - 1]));
            self.biases.append(np.random.randn(layers[layer_num]));
        """
        self.activation = string specifying what activation function the neurons will use. 
            The options are:
            sigmoid (default)
        self.cost_function = string specifying hat cost function will be used for the 
            network.
            The options are:
            MSE (default)
        """
        self.activation_function = activation_function;
        self.cost_function = cost_function;
 
    """
    TRAINING
    Train the network using stochastic gradient descent and backpropagation.
    Training data should be given in the following format:
        [x11, x12, ..., x1i, y1
         x21, x22, ..., x2i, y2
         ...
         xm1, xm1, ..., xmi, ym]
    Where each row corrsponds to a training example with i data points
    """
    def train(self, training_data, batch_size, num_epochs, learning_rate):
        for epoch in range(num_epochs):
            print("EPOCH: %d" % epoch);
            # Randomize the order of training examples
            np.random.shuffle(training_data);
            # Separate inputs from outputs
            inputs = training_data[:, :-1]
            outputs = training_data[:, -1];
            # For each epoch, loop through each batch to use as training data
            for batch in range(len(training_data))[::batch_size]:
                # For each batch, we calculate activations and use the backpropagation
                # algorithm to change the weights and biases using gradient descent
                self.Z = [];
                self.activations = [];
                # Create matrix out of all training inputs in the batch
                # If the first layer of the network has k neurons, and each training
                # example has i data points, then weights will be a kxi matrix 
                # so Wx_j = kx1 vector.
                # To apply W to all input vectors, we can multiply WX where
                # X is the ixm matrix containing all m training examples as columns
                X = inputs[batch : batch + batch_size];
                X = np.transpose(X);
                Y = outputs[batch : batch + batch_size];

                # *** DEBUGGING  ***
                print('Batch #%d' % batch);
                print('X');
                print(X);
                print('Y')
                print(Y);
                
                # FEEDFORWARD
                """
                self.Z[layer, neuron, training_example] = 
                                            List of vectors with weighted inputs to the neurons
                self.activations[layer, neuron, training_Example] = 
                                            List of vectors with activations for each neuron
                """
                # Calclate outputs going forwards through the network
                for layer in range(self.num_layers - 1):
                    if layer == 0:
                        # Feed inputs to the network
                        prev_activations = X;
                    else:
                        prev_activations = self.activations[layer - 1];
                    # Bias matrix where each column is a copy of the bias vector is needed
                    # to add bias terms for each training example. 
                    one_vector = np.ones(batch_size);
                    bias_matrix = np.outer(self.biases[layer], one_vector);
                    self.Z.append(np.dot(self.weights[layer], prev_activations) + bias_matrix);
                    self.activations.append(self.activation(self.Z[layer]));
                # BACKPROPAGATION
                # Calculate output error matrix so the [i, j]th entry contains the
                # error for the ith neuron in the output layer for the jth training
                # example
                output_error = np.multiply(
                                    self.cost_derivative(self.activations[-1], Y),
                                    self.activation_derivative(self.Z[-1]));
                # Backpropogate: we create the errors matrix which is indexed
                # in the form errors[layer, neuron, training_example]
                errors = [output_error];
                for layer in reversed(range(self.num_layers - 2)):
                    # For each layer, calculate errors in previous layer
                    previous_errors = np.multiply(
                                        np.dot(
                                            np.transpose(self.weights[layer + 1]),
                                            errors[0]), 
                                        self.activation_derivative(self.Z[-layer])); 
                    # Add previous errors to the beginning of the error matrix list
                    errors.insert(0, previous_errors);
                # Calculate gradint of cost function
                gradC_b = [];
                gradC_w = [];
                for layer in range(self.num_layers - 1):
                    # Gradient of cost wrt biases for a layer is just the
                    # vector of errors for that layer
                    gradC_b.append(np.average(errors[layer],1));
                    """
                    sum_of_weights[layer, j, k] will contain the partial derivative 
                    of cost wrt the weight from the kth neuron in layer - 1 to the jth
                    neuron in layer summed over all training examples. That is,
                    sum_of_weightes[layer, j, k] = [sum over training examples dC/dw_j,k]
                    """
                    if (layer == 0):
                        prev_activations = X;
                    else:
                        prev_activations = self.activations[layer - 1];
                    sum_of_weights = np.dot(
                                            errors[layer],
                                            np.transpose(prev_activations));
                    gradC_w.append((1 / batch_size) * sum_of_weights);               
                
                # DEBUGGING
                print("\nGradient wrt biases: ")
                print(gradC_b);
                print("\nGradient wrt weights: ")
                print(gradC_w);
                
                # Gradient Descent
                """
                At this point, gradC_b is of the form
                [layer, neuron] = average gradient for that neuron over all
                    training examples in the batch
                and gradC_w is of the form [layer, input_neuron, output_neuron], 
                    also averaged over all traininge examples in the batch
                """
                for layer in range(self.num_layers - 1):
                    self.biases[layer] -= learning_rate * gradC_b[layer];
                    self.weights[layer] -= learning_rate * gradC_w[layer];  
                
                
    """ 
    ACTIVATION FUNCTION
    For this network, we use the sigmoid function to calculate neuron activation
    In general, we assume the input z will be a matrix where the [i,j]th entry is the
    ith neuron in the jth training example
    """      
    def activation(self, z):
        if (self.activation_function == "sigmoid"):
            return 1.0 / (1 + np.exp(-z));
    
    def activation_derivative(self, z):
        if (self.activation_function == "sigmoid"):
            return np.multiply((1 - self.activation(z)), self.activation(z));
    """
    COST FUNCTION
    We assume that activations is a 2D matrix where each column corresponds to the activations
    for a specific training example and each row corresponds to a specific neuron

    We assume for now that outputs for the network are disjoint categories, so only
    one output neuron should fire at a time. the output_matrix function turns the output
    vector where each entry corresponds to a training example into 
    
    Both calculate_cost and cost_derivative return a row vector where the ith entry
    is the cost/cost derivative for training example i
    """
    def output_matrix(self, activations, outputs):
        # output_matrix[neuron, training_example]
        output_matrix = np.zeros((activations.shape[0], outputs.shape[0]));
        for training_example, output in enumerate(outputs):
            output_matrix[output, training_example] = 1;
        return output_matrix;
    
    def calculate_cost(self, activations, outputs):
        outputs_matrix = self.output_matrix(activations, outputs);
        if (self.cost_function == "MSE"):
            squared_errors = np.square(activations - output_matrix);
            # Average sum of squared errors over each training example
            return 0.5 * np.average(np.sum(squared_errors, 0));
    """
    Derivative of cost functions with respect to activations
    Each entry [i, j] in the resulting matrix will be the gradient of the cost function
    with respect to the activation of the ith neuron in the jth training example
    """
    def cost_derivative(self, activations, outputs):
        output_matrix = self.output_matrix(activations, outputs);
        if (self.cost_function == "MSE"):
            return (activations - output_matrix);
       
    """ 
    TODO: 
        test(testing_data)
        predict(data)
    """
    def print_network(self):
        print("Number of layers: ");
        print(self.num_layers);
        print("Weights: ")
        for layer in self.weights:
            print(layer)
        print("\nBiases:" )
        for layer in self.biases:
            print(layer)
        print("\nWeighted Inputs:")
        for layer in self.Z:
            print(layer)
        print("\nActivations:")
        for layer in self.activations:
            print(layer);

In [31]:
"""
TEST NETWORK CREATION
"""
test = Simple_NN([3, 6, 4]);
random_data = np.matrix('1, 2, 3, 0; 11, 12, 13, 1; 21, 22, 23, 2; 31, 32, 33, 3')
batch_size = 2;
num_epochs = 1;
learning_rate = 0.5;
test.train(random_data, batch_size, num_epochs, learning_rate);

print("After training");
test.print_network();



EPOCH: 0
Batch #0
X
[[ 1 11]
 [ 2 12]
 [ 3 13]]
Y
[[0]
 [1]]

Gradient wrt biases: 
[matrix([[-0.004144  ],
        [ 0.0209995 ],
        [-0.00144362],
        [ 0.00471528],
        [-0.00010283],
        [ 0.00029675]]), matrix([[-0.03533802],
        [-0.03470577],
        [ 0.07828954],
        [ 0.00748207]])]

Gradient wrt weights: 
[matrix([[-0.00861586, -0.01275986, -0.01690385],
        [ 0.0209976 ,  0.0419971 ,  0.0629966 ],
        [-0.00144362, -0.00288724, -0.00433086],
        [ 0.00471534,  0.00943063,  0.01414591],
        [-0.00010101, -0.00020383, -0.00030666],
        [ 0.00029675,  0.00059349,  0.00089024]]), matrix([[ -3.24009286e-02,  -1.30294731e-02,  -8.03356125e-04,
          -3.37070153e-02,  -3.51273862e-02,  -3.50971298e-02],
        [ -3.25804265e-02,   1.10101859e-03,   6.78959149e-05,
           2.84869000e-03,  -3.47229395e-02,  -3.47261252e-02],
        [  7.29169854e-02,   8.10037048e-03,   4.99427414e-04,
           2.09549548e-02,   7.81576569e-02

ValueError: non-broadcastable output operand with shape (6,) doesn't match the broadcast shape (6,6)